In [ ]:
# Slow method

In [ ]:
import random
import numpy as np
import networkx as nx

In [ ]:
def generate_transtions_v0 (num_nodes , num_attractors):
    n = num_nodes
    s = 2**n # number of states
    r = num_attractors

    num_att = 0
    while num_att != r:
        edgelist = [(i,random.randint(0, s-1)) for i in range(s)]
        G = nx.from_edgelist(edgelist)
        num_att = nx.number_connected_components(G)
    
    sizes = [len(B) for B in sorted(nx.connected_components(G), key=len, reverse=True)]

    return sizes, edgelist

In [ ]:
basin_sizes, transitions = generate_transtions_v0 (num_nodes = 10 , num_attractors = 5)
basin_sizes

In [ ]:

# Faster method

In [ ]:
import random
import networkx as nx

In [ ]:
def transitions_between_layers(inner_layer,outher_layer):
    """
    Generate transitions from the outher to the inner layer.
    The inputs are lists of states in the two layers
    """
    edges = []
    for i in outher_layer:
        random.shuffle(inner_layer)
        j = inner_layer[0]
        edges.append((i,j))
    return edges

In [ ]:
def core_transitions(length):
    """
    Given the length of the cycle,
    generate its associated transitions.
    These states take labels 0, 1, ..., length 
    """
    return [(i, (i+1)%length) for i in range(length)]

In [ ]:
def layer_distribution(length, num_states):
    """
    Generate the distribution of the number of states,
    [n1,n2,n3,...]
    where n1 is the number of states in the innermost layer (i.e. the cycle),
    n2 is the number of states in layer 2, etc.
    """
    layer_widths = [length]
    while sum(layer_widths) < num_states:
        assigned_states = sum(layer_widths)
        m = random.randint(1,num_states-assigned_states)
        layer_widths.append(m)
    return layer_widths

In [ ]:
# generate the distribution of states

def transitions(length, num_states):
    """
    Generate the list of transitions for a single connected component.
    The states are labeled 0, ..., num_states.
    """
    
    dist = layer_distribution(length, num_states)
    edges = core_transitions(length)

    for i in range(len(dist)-1):
    
        # outher_layer
        layer_id = i+1
        start = sum(dist[:layer_id])
        stop  = sum(dist[:layer_id]) + dist[layer_id]
        outher_layer = list(range(start, stop))
    
        # inner_layer
        layer_id = i
        start = sum(dist[:layer_id])
        stop  = sum(dist[:layer_id]) + dist[layer_id]
        inner_layer = list(range(start, stop))
    
        edges = edges + transitions_between_layers(inner_layer,outher_layer)
    
    return edges

In [ ]:
edges = transitions(length = 3, num_states = 100)
edges[:10]

In [ ]:
G = nx.from_edgelist(edges)
pos = nx.spring_layout(G)
nx.draw_networkx(G, pos)

In [ ]:
G = nx.from_edgelist(edges)
num_att = nx.number_connected_components(G)
num_att

In [ ]:
# To do:
# Now I need to separate the larger transition graph into connected components generated this way

# I can either generate all transitions, or just select certain inputs, 
# evolve those, then randomly assign the global state labels

In [ ]:
def join_transitions(transitions1,transitions2):
    """
    Joins two lists of transitions after shifting the labels in the second list
    """
    s1 = len(transitions1)
    s2 = len(transitions2)
    #shift the labels in transitions2
    p = list(range(s1,s1+s2))
    new_transitions2 = [(p[i], p[j]) for (i, j) in transitions2]
    return transitions1 + new_transitions2

In [ ]:
def labels_permutation (transitions):
    """
    randomly reassign state labels
    """
    # Generate a random permutation of the labels
    s = len(transitions)
    p = list(range(s))
    random.shuffle(p)
    # Create a new list of edges with the updated labels
    new_transitions = [(p[i], p[j]) for (i, j) in transitions]
    sorted_transitions = sorted(new_transitions, key=lambda x: x[0])
    return sorted_transitions

In [ ]:
s1 = 2**8
s2 = 2**4
s3 = 2**4
t1 = transitions(4, s1)
t2 = transitions(2, s2)
t3 = transitions(1, s3)
#print(t1)
#print(t2)
s1+s2+s3

In [ ]:
t = join_transitions( join_transitions(t1,t2) , t3)
edges = labels_permutation (t)

In [ ]:
G = nx.from_edgelist(edges)
nx.draw_networkx(G)

In [ ]:
def attractor_landscape():
    """
    """
    ...
    return ...

In [ ]:
n = 3
labels = list(range(2**n))


num_nodes = 10
num_states = 2**num_nodes

landscape = [[3,1],[1,2],[1,5]]
# this describes a transition graph with 3 attractors: a cycle with length 3, and two fixed points.
# the second element in each entry is the relative size of the basin. 
# In this case the basin sizes are in ratios 1:2:5

sum(np.array(landscape)[:,1])

In [ ]:
n = 16
s = 2**n
s

In [ ]:
b1 = .25 # relative size of basin 1
b2 = .50 # relative size of basin 2
b3 = .05 # relative size of basin 3
b4 = .20 # relative size of basin 3

b1+b2+b3+b4

In [ ]:
s1 = int(b1*s)
s2 = int(b2*s)
s3 = int(b3*s)

s4 = s-s1-s2-s3

In [ ]:
print(s1)
print(s2)
print(s3)
print(s4)

In [ ]:
s-(s1+s2+s3+s4)

In [ ]:
length1 = 4
length2 = 1
length3 = 1
length4 = 5


t1 = transitions(length1, s1)
t2 = transitions(length2, s2)
t3 = transitions(length3, s3)
t4 = transitions(length4, s4)


In [ ]:
t = join_transitions(join_transitions( join_transitions(t1,t2) , t3) , t4)
edges = labels_permutation (t)

In [ ]:
edges

In [ ]:
# this method is too slow for networks of reasonable size.
# write a version with sampling